In [28]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


df = pd.read_csv('dataset.csv')
df[df['class'] == 'slouch'] #Grabs only a specific class.

    

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
641,slouch,0.541791,0.590731,-1.691134,0.998342,0.566436,0.526417,-1.638459,0.998234,0.581153,...,0.381728,0.018999,0.599024,2.641183,0.389301,0.019661,0.473942,2.689452,0.045508,0.029838
642,slouch,0.541812,0.591890,-1.325245,0.998551,0.566528,0.528817,-1.276093,0.998443,0.581142,...,0.285167,0.018287,0.602644,2.733079,0.321510,0.018357,0.477484,2.765494,-0.006464,0.027641
643,slouch,0.541852,0.591780,-1.327400,0.998620,0.566590,0.528874,-1.269794,0.998524,0.581143,...,0.285119,0.018644,0.598133,2.725699,0.330814,0.018355,0.472494,2.765246,-0.007753,0.027917
644,slouch,0.541774,0.591748,-1.321185,0.998693,0.566203,0.528913,-1.264028,0.998612,0.580568,...,0.209224,0.019046,0.590351,2.747096,0.218334,0.018740,0.468462,2.771281,-0.111675,0.028259
645,slouch,0.541714,0.590576,-1.277558,0.998709,0.566106,0.528277,-1.225650,0.998638,0.580319,...,0.269662,0.019711,0.586190,2.728146,0.254174,0.020057,0.466422,2.726785,-0.016072,0.030151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1630,slouch,0.414760,0.545447,-1.822086,0.999552,0.425885,0.453281,-1.783790,0.999496,0.438941,...,0.333178,0.013642,0.659191,3.129954,0.455932,0.011867,0.505490,3.218334,-0.053058,0.015615
1631,slouch,0.415396,0.542980,-1.819880,0.999562,0.426485,0.451537,-1.773503,0.999504,0.439641,...,0.274269,0.013389,0.658969,3.130836,0.350598,0.011458,0.506565,3.229891,-0.124930,0.015086
1632,slouch,0.415594,0.541222,-1.844805,0.999577,0.426971,0.450639,-1.796209,0.999517,0.440175,...,0.278684,0.012945,0.660680,3.135844,0.342882,0.010971,0.508591,3.240272,-0.120705,0.014411
1633,slouch,0.417615,0.536509,-1.808223,0.999603,0.429062,0.446675,-1.756533,0.999544,0.442238,...,0.261497,0.012472,0.667265,3.158958,0.277279,0.010512,0.517116,3.254115,-0.135031,0.013666


In [29]:
X = df.drop('class', axis=1) #Features (no class col)
y = df['class'] #Target value (class col)


In [30]:
#Create train/ test split.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

# Train Classification Model

In [31]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [32]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

#Train every model (4) on the data.
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [34]:
fit_models['lr'].predict(X_test)

array(['upright', 'slouch', 'upright', 'slouch', 'slouch', 'slouch',
       'upright', 'slouch', 'upright', 'upright', 'slouch', 'slouch',
       'slouch', 'upright', 'slouch', 'upright', 'slouch', 'upright',
       'slouch', 'slouch', 'slouch', 'slouch', 'slouch', 'slouch',
       'upright', 'slouch', 'slouch', 'upright', 'upright', 'upright',
       'slouch', 'slouch', 'slouch', 'slouch', 'slouch', 'slouch',
       'upright', 'slouch', 'upright', 'upright', 'upright', 'slouch',
       'slouch', 'upright', 'slouch', 'upright', 'upright', 'upright',
       'upright', 'slouch', 'slouch', 'slouch', 'upright', 'slouch',
       'slouch', 'slouch', 'upright', 'slouch', 'slouch', 'slouch',
       'slouch', 'slouch', 'upright', 'slouch', 'slouch', 'slouch',
       'upright', 'upright', 'slouch', 'slouch', 'upright', 'slouch',
       'upright', 'slouch', 'slouch', 'slouch', 'slouch', 'slouch',
       'upright', 'upright', 'upright', 'slouch', 'upright', 'upright',
       'slouch', 'slouch', 'u

# Evaluate & Serialise Models

In [35]:
from sklearn.metrics import accuracy_score
import pickle

In [36]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))


lr 1.0
rc 1.0
rf 0.9979633401221996
gb 0.9979633401221996


In [38]:
# SAVE MODEL #
with open('pose_detection.pkl', 'wb') as f:
    pickle.dump(fit_models['lr'], f)

# Make Detections w/ Model

In [ ]:
# RELOAD INTO NOTEBOOK #
with open('pose_detection.pkl', 'rb') as f:
    model = pickle.load(f)

